# Predictions blender

In [1]:
# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
from glob import glob
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

import shutil

# Project
project_common_path = os.path.dirname('.')
project_common_path = os.path.abspath(os.path.join(project_common_path, '..', 'common'))
if not project_common_path in sys.path:
    sys.path.append(project_common_path)

## Simple probability mean 

In [12]:
import numpy as np
import pandas as pd

from data_utils import get_id_type_list_for_class, GENERATED_DATA, OUTPUT_PATH
from test_utils import create_submission

In [6]:
predictions_csv = glob(os.path.join(OUTPUT_PATH, "*.csv"))
predictions_csv

['D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_0_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_10_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_11_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_12_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_13_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_14_SqueezeNet21_all_classes_fold=4_seed=20172017-07-10-10-18.csv',
 'D:\\vfomin\\MyExamples\\python\\planet_amazon_rainforest\\output\\predictions_1_SqueezeNet21_all_classes_fold=4_seed=

In [7]:
predictions = []
for csv_filepath in predictions_csv:
    predictions.append(pd.read_csv(csv_filepath))

In [18]:
predictions[0].sort_values(by='image_name').head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
12574,file_0,0.066559,0.026386,0.022647,0.027997,0.013708,0.943612,0.019972,0.032191,0.064726,0.007336,0.038537,0.124131,0.932207,0.030553,0.022448,0.060615,0.021036
55285,file_1,0.038082,0.014214,0.010752,0.015895,0.007087,0.972440,0.007770,0.019515,0.043018,0.002502,0.017839,0.100316,0.956652,0.018208,0.010303,0.034832,0.010925
8330,file_10,0.531736,0.088515,0.147094,0.066210,0.053297,0.578340,0.081342,0.103375,0.219836,0.139833,0.157908,0.314950,0.895389,0.175684,0.085034,0.094970,0.237179
19435,file_100,0.180771,0.013756,0.018632,0.009134,0.013630,0.951660,0.010162,0.021247,0.061058,0.009143,0.032453,0.117525,0.971390,0.055973,0.015998,0.061598,0.047556
54655,file_1000,0.244277,0.051670,0.078124,0.059738,0.039378,0.754265,0.064522,0.097159,0.168164,0.050973,0.091554,0.297560,0.866799,0.116241,0.045633,0.082810,0.119394


In [19]:
predictions[1].sort_values(by='image_name').head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
16525,file_0,0.073166,0.029202,0.025499,0.028145,0.016391,0.949343,0.018040,0.036076,0.075053,0.007552,0.037240,0.133683,0.930354,0.032100,0.023555,0.064420,0.022039
60335,file_1,0.030922,0.017255,0.010863,0.015595,0.008960,0.966943,0.008244,0.024155,0.054117,0.002726,0.022738,0.113144,0.956747,0.015055,0.012653,0.040333,0.009813
38579,file_10,0.313379,0.051428,0.100774,0.047588,0.058047,0.721346,0.031028,0.072293,0.221484,0.057846,0.107129,0.291878,0.893824,0.121176,0.054719,0.104063,0.139087
21391,file_100,0.151895,0.045386,0.058039,0.066351,0.020819,0.876061,0.031993,0.077704,0.123705,0.029359,0.056405,0.235682,0.886710,0.076777,0.047129,0.048100,0.066734
24228,file_1000,0.479737,0.062571,0.088518,0.045429,0.046464,0.707323,0.058361,0.065683,0.153167,0.063890,0.158736,0.298752,0.882938,0.228939,0.037973,0.101017,0.173090


In [8]:
def compute_mean(predictions):
    df = predictions[0]
    for p in predictions[1:]:
        df = pd.concat([df, p], axis=0)
    df = df.apply(pd.to_numeric, errors='ignore')
    gb = df.groupby('image_name')
    df2 = gb.agg(np.mean).reset_index()
    return df2

In [9]:
mean_predictions_df = compute_mean(predictions)

In [10]:
mean_predictions_df.head()

,image_name,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,file_0,0.075577,0.022042,0.028876,0.018089,0.032277,0.903471,0.013046,0.059221,0.093378,0.027963,0.020402,0.083580,0.967447,0.053680,0.035005,0.046575,0.088839
1,file_1,0.035373,0.011506,0.014149,0.007825,0.020209,0.938606,0.004963,0.037624,0.064372,0.013600,0.006733,0.056471,0.984725,0.029025,0.024612,0.022560,0.068315
2,file_10,0.449505,0.034339,0.059886,0.027366,0.057424,0.431175,0.063071,0.088553,0.169601,0.047239,0.087217,0.583021,0.913226,0.166780,0.118563,0.112523,0.232856
3,file_100,0.078833,0.020902,0.019649,0.012992,0.025159,0.939492,0.007644,0.047085,0.070833,0.030681,0.010401,0.059309,0.973639,0.079619,0.026858,0.032322,0.110833
4,file_1000,0.281819,0.043391,0.059090,0.035357,0.050177,0.832329,0.022377,0.087638,0.141859,0.083981,0.046980,0.133932,0.937326,0.180613,0.047837,0.055389,0.167381


In [11]:
thresholds = {
    'agriculture': 0.35,
    'artisinal_mine': 0.35,
    'bare_ground': 0.35,
    'blooming': 0.35,
    'blow_down': 0.1,
    'clear': 0.5,
    'cloudy': 0.35,
    'conventional_mine': 0.35,
    'cultivation': 0.35,
    'habitation': 0.35,
    'haze': 0.35,
    'partly_cloudy': 0.35,
    'primary': 0.5,
    'road': 0.35,
    'selective_logging': 0.35,
    'slash_burn': 0.1,
    'water': 0.35
}

In [14]:
create_submission(mean_predictions_df, info="squeezenet21_mean_5_folds", thresholds=thresholds)